In [ ]:
import pandas as pd
import pyodbc
import re

In [ ]:
trans_type_list = { 
    'deposits' : [
        'DIRECT DEPOSIT',
        'DEPOSIT', 
        'M&T ATM CHECK DEPOSIT',
        'M&T ATM CASH DEPOSIT',
        'MOBILE DEPOSIT',
        'VISA PURCHASE CREDIT'
    ],

    'purchases_payments' : [
        'CHECK PAID',
        'VISA PURCHASE',
        'CARD PURCHASE',
        'VISA PURCHASE (RECURRING)',
        'VISA PIN PURCHASE',
        'PURCHASE WITH CASH BACK',
        'VISA PIN PURCHASE WITH CASH BACK',
        'WEB BILL PAYMENT FROM CHECKING',
        'WEB PAYMENT TO CREDIT CARD',
        'WEB PAYMENT',
        'ACH MISCELLANEOUS DEBIT',
        'AUTO PAYMENT TO M&T BANK'    
    ],

    'withdrawals' : [
        'PREAUTHORIZED WITHDRAWAL',
        'QUICK BANK WITHDRAWAL',
        'ATM WITHDRAWAL',
        'COUNTER WITHDRAWAL'  
    ],

    'transfers' : [
        'WEB TRANSFER TO CHECKING',
        'WEB TRANSFER FROM CHECKING',
        'VISA MONEY TRANSFER',
        'TELLER TRANSFER CREDIT',
        'INCOMING FEDWIRE FUNDS TRANSFER        &',
        'TELLER TRANSFER DEBIT',
        'QUICKBANK TXF FROM SAV TO DDA',
        r'ZELLE P2P PAYMENT SENT                 \\',
        r'ZELLE P2P PAYMENT RECEIVED             \\',
        'CREDIT LINE TRANSFER-M&T BANK'    
    ],

    'fees' : [
        'NON-M&T ATM FEE',
        'INTERNATIONAL TRANSACTION FEE',
        'WAIVE ADD-ON FEE: NON M&T ATM FEE WAIVER',
        'INSUFFICIENT FUNDS FEE',
        'M&T ADD-ON FEE: NON M&T ATM FEE REFUND',
        'WAIVE ADD-ON FEE: NON M&T ATM FEE REFUND',
        'M&T ADD-ON FEE: CHKS/SAFE DEPOSIT BOX',
        'WAIVE ADD-ON FEE: CHKS/SAFE DEPOSIT BOX',
        'M&T ADD-ON FEE: NON M&T ATM FEE WAIVER',
        'EFT SERVICE CHARGE',
        'NON-M&T ATM FEE REV',
        'M&T ADD-ON FEE: OVERDRAFT TRANSFERS',
        'WAIVE ADD-ON FEE: OVERDRAFT TRANSFERS',
        'BUNDLE FEE WAIVER',
        'OVERDRAFT BALANCE EXCESS FEE',
        'SERVICE CHARGE',
        'ATM SURCHARGE REBATE',
        'CHECK VENDOR CHARGE',
        'REVERSE MONTHLY SERVICE CHARGE',
        'REVERSE INSUFFICIENT FUNDS FEE',
        'MONTHLY SERVICE CHARGE',
        'SERVICE CHG WAIVE- RELATIONSHIP PRICING'    
    ],

    'misc' : [
        'ACH CONVERTED CHECK',
        'OVERDRAFT TRANSFER FROM CREDIT CARD',
        'FEDERAL WITHHOLDING ON INTEREST PAYMENT',
        'OD PROTECTION FUND TRANSFER',
        'REVERSE CARD PURCHASE',
        'VISA PIN CREDIT VOUCHER/MERCHANDISE RET',
        'DEBIT CARD DISPUTE PROVISIONAL CREDIT',
        'NSF ITEM PRESENTED NOTATION',
        'CLOSEOUT',
        'INTEREST PAYMENT'  
    ]
}

load_order = [1,2]

metrics = ['transaction_amt', 'trans_post_ledger_bal_amt']

#string and lists to build
loop_string = ''
int_cols = []
float_cols = []

#create transaction counts
for j in load_order:
    loop_string += ",sum(case when cal1.load_order = {0} then 1 else 0 end) as trans_count_total_{0}".format(j)
    int_cols.append("trans_count_total_{0}".format(j))
    while j+1 < 3:
        loop_string += ",sum(case when cal1.load_order = {0} then 1 else 0 end) - sum(case when cal1.load_order = {1} then 1 else 0 end) as trans_count_total_dif".format(j,j+1)
        int_cols.append("trans_count_total_dif".format(j,j+1))
        break
    for i in trans_type_list:
        loop_string += ",sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {1} then 1 else 0 end) as trans_count_{2}_{1}".format(','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip())
        int_cols.append("trans_count_{1}_{0}".format(j,re.sub(r'\W+', '_', i).strip()))
        while j+1 < 3:
            loop_string += ",sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {1} then 1 else 0 end) - sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {3} then 1 else 0 end) as trans_count_{2}_dif".format(','.join(["'"+l+"'" for l in trans_type_list[i]])  ,j,re.sub(r'\W+', '_', i).strip(),j+1)
            int_cols.append("trans_count_{0}_dif".format(re.sub(r'\W+', '_', i).strip()))
            break

#create averaging metrics
for j in load_order:
    for k in metrics:        
        loop_string += ",avg(case when cal1.load_order = {0} then dt.{1} else 0 end) as {1}_{0}".format(j,k)
        float_cols.append("{1}_{0}".format(j,k))
        while j+1 < 3:
            loop_string += ",avg(case when cal1.load_order = {0} then dt.{2} else 0 end) - sum(case when cal1.load_order = {1} then dt.{2} else 0 end) as {2}_dif".format(j,j+1,k)
            float_cols.append("{0}_dif".format(k))
            break
        for i in trans_type_list:
            loop_string += ",avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {2} then dt.{0}  else 0 end) as {0}_{3}_{2}".format(k,','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip())
            float_cols.append("{0}_{2}_{1}".format(k,j,re.sub(r'\W+', '_', i).strip()))
            while j+1 < 3:
                loop_string += ",avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {2} then dt.{0}  else 0 end) - avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {4} then dt.{0}  else 0 end) as {0}_{3}_dif".format(k,','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip(),j+1)
                float_cols.append("{0}_{1}_dif".format(k,re.sub(r'\W+', '_', i).strip()))
                break

In [ ]:
print(loop_string)

### Define Query

In [ ]:
queryfromProd = r"""

removed

""".format(loop_string)

In [ ]:
#don't chnage the connection string
prod_connect_string =  'removed'                        
                        
connProd = pyodbc.connect(prod_connect_string,autocommit=True)

In [ ]:
%timeit
write_header = True
directory = r'data'
file = r'\deposit_data_full_condensed.csv'
df = pd.read_sql(queryfromProd, connProd, chunksize= 10000)
for chunk in df:
    chunk.to_csv(directory + file, 
                              sep=',', 
                              index = False,
                              header= write_header,
                              encoding = 'utf-8',
                              mode='a')
    write_header = False

In [ ]:
df_partial_import = pd.read_csv(directory+file, dtype={**{i: 'int32' for i in int_cols} , **{i: 'float32' for i in float_cols}})

In [ ]:
df_partial_import

In [ ]:
df_partial_import.closed_within_next_month.mean()

### Look at data to make sure it's what we expect

In [ ]:
def df_stats(df):
    print(df.dtypes)
    print('_'*100)
    # dataframe.size 
    size = df.size 

    # dataframe.shape 
    shape = df.shape 

    # dataframe.ndim 
    df_ndim = df.ndim 

    #df mem usage
    df_memu = df.memory_usage() 


    # printing size and shape 
    print("Size = {}\n\
    Shape = {}\n\
    Shape[0] x Shape[1] = {}\n\
    Memory Usage = \n{}". 
    format(size, shape, shape[0]*shape[1], df_memu)) 

In [ ]:
df_stats(df_partial_import)

In [ ]:
df_partial_import.to_pickle(r'data/deposit_data_consensed.pkl')